# Tool recommendation 
## (Dense neural network with weighted cross-entropy loss)

In [11]:
import numpy as np
import json
import warnings
import operator

import h5py
from keras.models import model_from_json
from keras import backend as K

warnings.filterwarnings("ignore")


def read_file(file_path):
    with open(file_path, 'r') as data_file:
        data = json.loads(data_file.read())
    return data


def create_model(model_path):
    reverse_dictionary = dict((str(v), k) for k, v in dictionary.items())
    model_weights = list()
    weight_ctr = 0
    for index, item in enumerate(trained_model.keys()):
        if "weight_" in item:
            d_key = "weight_" + str(weight_ctr)
            weights = trained_model.get(d_key).value
            model_weights.append(weights)
            weight_ctr += 1
    # set the model weights
    loaded_model.set_weights(model_weights)
    return loaded_model, dictionary, reverse_dictionary


def compute_recommendations(model, tool_sequence, labels, dictionary, reverse_dictionary, class_weights, topk=20, max_seq_len=25):
    tl_seq = tool_sequence.split(",")
    last_tool_name = reverse_dictionary[str(tl_seq[-1])]
    sample = np.zeros(max_seq_len)
    for idx, tool_id in enumerate(tl_seq):
        sample[idx] = int(tool_id)
    sample_reshaped = np.reshape(sample, (1, max_seq_len))
    tool_sequence_names = [reverse_dictionary[str(tool_pos)] for tool_pos in tool_sequence.split(",")]
    # predict next tools for a test path
    prediction = model.predict(sample_reshaped, verbose=0)
    weight_val = list(class_weights.values())
    weight_val = np.reshape(weight_val, (len(weight_val),))
    prediction = np.reshape(prediction, (prediction.shape[1],))
    prediction = prediction * weight_val
    prediction = prediction / float(np.max(prediction))
    prediction_pos = np.argsort(prediction, axis=-1)
    # get topk prediction
    topk_prediction_pos = prediction_pos[-topk:]
    topk_prediction_val = [int(prediction[pos] * 100) for pos in topk_prediction_pos]
    # read tool names using reverse dictionary
    pred_tool_ids = [reverse_dictionary[str(tool_pos)] for tool_pos in topk_prediction_pos]
    pred_tool_ids_sorted = dict()
    for (tool_pos, tool_pred_val) in zip(topk_prediction_pos, topk_prediction_val):
        tool_name = reverse_dictionary[str(tool_pos)]
        pred_tool_ids_sorted[tool_name] = tool_pred_val
    pred_tool_ids_sorted = dict(sorted(pred_tool_ids_sorted.items(), key=lambda kv: kv[1], reverse=True))
    ids_tools = dict()
    keys = list(pred_tool_ids_sorted.keys())
    tool_seq_name = ",".join(tool_sequence_names)
    print("Current tool sequence: ")
    print()
    print(tool_seq_name)
    print()
    print("Recommended tools for the tool sequence '%s' with their scores in decreasing order:" % tool_seq_name)
    print()
    for i in pred_tool_ids_sorted:
        print(i + "(" + str(pred_tool_ids_sorted[i]) + "%)")
    for key in pred_tool_ids_sorted:
        ids_tools[key] = dictionary[key]
    print()
    print("Tool ids:")
    print()
    for i in ids_tools:
        print(i + "(" + str(ids_tools[i]) + ")")

## Unpack trained model for prediction

In [12]:
model_path = "data/tool_recommendation_model.hdf5"
trained_model = h5py.File(model_path, 'r')
model_config = json.loads(trained_model.get('model_config').value)
dictionary = json.loads(trained_model.get('data_dictionary').value)
class_weights = json.loads(trained_model.get('class_weights').value)
loaded_model = model_from_json(model_config)

model, dictionary, reverse_dictionary = create_model(model_path)

## Indices of tools

In [13]:
print(reverse_dictionary)

{'1': 'EMBOSS: geecee41', '2': 'augustus', '3': 'peptide_shaker', '4': 'melt', '5': 'export2graphlan', '6': 'flaimapper', '7': 'gd_raxml', '8': 'gmx_nvt', '9': 'passatutto', '10': 'mothur_homova', '11': 'chipeakanno_annopeaks', '12': 'bamCoverage_deepTools', '13': 'kraken-filter', '14': 'extract_bcs.py', '15': 'tp_grep_tool', '16': 'feelnc', '17': 'annotateMyIDs', '18': 'ggplot2_histogram', '19': 'EMBOSS: wordcount109', '20': 'idpqonvert', '21': 'metaquantome_stat', '22': 'flexbar_no_split', '23': 'fastp', '24': 'prepare_box', '25': 'methtools_plot', '26': 'bismark_bowtie', '27': 'smf_utils_find-boxes', '28': 'blastxml_to_tabular_selectable', '29': 'query_tabular', '30': 'blockbuster', '31': 'EMBOSS: transeq101', '32': 'sailfish', '33': 'tophat_fusion_post', '34': 'tp_cut_tool', '35': 'deeptools_bigwigCompare', '36': 'pureclip', '37': 'regionalgam_ab_index', '38': 'proteomics_search_tandem_1', '39': 'vigiechiro_idvalid', '40': 'convert_bc_to_binary_RY.py', '41': 'scanpy_run_tsne', '42'

## Recommended tools

In [14]:
topk = 20 # set the maximum number of recommendations
tool_seq = "494" # give tools ids in a sequence and see the recommendations. To know all the tool ids, 
                     # please print the variable 'reverse_dictionary'
compute_recommendations(model, tool_seq, "", dictionary, reverse_dictionary, class_weights, topk)

Current tool sequence: 

rna_star

Recommended tools for the tool sequence 'rna_star' with their scores in decreasing order:

fastqc(100%)
tophat2(22%)
bedtools_multicovtbed(0%)
freebayes(0%)
Extract genomic DNA 1(0%)
trinity(0%)
cutadapt(0%)
bwa(0%)
spades(0%)
IDPosteriorErrorProbability(0%)
sam_to_bam(0%)
snpEff(0%)
stringtie(0%)
snpSift_filter(0%)
trimmomatic(0%)
__FLATTEN__(0%)
bowtie2(0%)
rna_star(0%)
fastq_to_fasta_python(0%)
multiqc(0%)

Tool ids:

fastqc(538)
tophat2(477)
bedtools_multicovtbed(856)
freebayes(1122)
Extract genomic DNA 1(1050)
trinity(1020)
cutadapt(75)
bwa(824)
spades(748)
IDPosteriorErrorProbability(236)
sam_to_bam(230)
snpEff(213)
stringtie(363)
snpSift_filter(79)
trimmomatic(335)
__FLATTEN__(758)
bowtie2(1172)
rna_star(494)
fastq_to_fasta_python(1070)
multiqc(309)
